In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
#temp = pd.read_csv('embeddings/250_0.16_vectors.tsv', sep=' ', header=None, skiprows=1)
temp = pd.read_csv("/h/224/stevenhl/chess_dim_skill/src/word2vecf/vectors/size_alpha/vecs_0.16_265.txt", sep=' ', header=None, skiprows=1)
temp = temp.drop(len(temp.columns)-1,axis=1)
#problem_id = list(temp[0])
temp.rename(columns={0:'puzzle_id'}, inplace=True)
temp.set_index('puzzle_id',inplace=True)
temp = temp.divide(np.linalg.norm(temp, axis=1), axis=0)

In [3]:
tag_data = pd.read_csv('/w/225/1/chess/tactics/tactics_problem.csv')
def get_first_tag(row):
    tags = str(row['tags']).split(',')
    return tags[0]

tag_data['tags'] = tag_data.apply(lambda row: get_first_tag(row),axis=1)
#tag_data = tag_data.loc[(tag_data['tags'] != 'nan') & (tag_data['tags'] != 'Mate in 1')]
#tag_data = tag_data.loc[(tag_data['tags'] != 'nan')]
#tag_data = tag_data.loc[tag_data['tags'].isna() == False]

In [4]:
tag_data = tag_data.loc[(tag_data['tags'] != 'nan')]
tag_data

,tactics_problem_id,rating,rd,attempt_count,average_seconds,move_count,tags,pgn
0,731,1569,44.96,53336,41,3,Attacking f7/f2,"[Event ""?""]\n[Site ""?""]\n[Date ""????.??.??""]\n..."
1,732,1417,45.44,63725,26,2,Basic Checkmates,"[Event ""?""]\n[Site ""?""]\n[Date ""????.??.??""]\n..."
2,738,2443,44.59,18115,54,3,Back Rank,"[Event ""?""]\n[Site ""?""]\n[Date ""????.??.??""]\n..."
3,739,1795,44.97,41674,57,3,Discovered Attack,"[Event ""?""]\n[Site ""?""]\n[Date ""????.??.??""]\n..."
4,740,2148,45.14,25701,88,5,Discovered Attack,"[Event ""?""]\n[Site ""?""]\n[Date ""????.??.??""]\n..."
...,...,...,...,...,...,...,...,...
54799,661054,2019,62.93,1473,97,7,Mate in 3+,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat..."
54800,661056,1909,65.29,1545,89,4,Decoy / Deflection,"[Event ""ch-USA 2018""]\n[Site ""Saint Louis USA""..."
54801,661058,2364,79.76,470,84,4,Mate in 3+,"[Event ""Blitz (3-minute)""]\n[Site ""Chess.com""]..."
54802,661060,2526,93.04,310,88,2,Hanging Piece,"[Event ""?""]\n[Site ""?""]\n[Date ""????.??.??""]\n..."


In [5]:
shared_puzzles = pd.merge(tag_data,temp,left_on='tactics_problem_id',right_on='puzzle_id')['tactics_problem_id']

In [25]:
#Sample tags and generate distance matrix of those tags
import sklearn.metrics
sample_size=1000
np.random.seed(100)
sampled_problems = np.random.choice(shared_puzzles.values, size = sample_size, replace = False)
#sampled_problems = np.array(shared_puzzles)
def generate_matrix(sampled_problems):
    vecs = temp.loc[sampled_problems]
    cos_sim = sklearn.metrics.pairwise.cosine_similarity(vecs)
    cos_dist = 1.0 - cos_sim
    #Set really small numbers to 0
    cos_dist=cos_dist.round(3)
    #Create cosine distance table
    table = pd.DataFrame(cos_dist, columns=sampled_problems,index=sampled_problems)
    return table

In [26]:
#Do clustering on condendesed distance matrix
table = generate_matrix(sampled_problems)
import scipy.spatial as sp, scipy.cluster.hierarchy as hc
linkage = hc.linkage(sp.distance.squareform(table), method='complete')

In [27]:
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist

c, coph_dists = cophenet(linkage,sp.distance.squareform(table))
c

0.6320976028181126

In [28]:
from operator import itemgetter
from scipy.cluster.hierarchy import fcluster
k=20
cluster = fcluster(linkage, 20, criterion='maxclust')
prob_pair = list(zip(list(sampled_problems), list(cluster)))
#prob_pair_sorted = sorted(prob_pair, key=itemgetter(1))

#problem_id_grouped = [x[0] for x in prob_pair_sorted]
#new_table = generate_matrix(problem_id_grouped)

In [29]:
cluster_to_id = {}
for i in range(1,21):
    cluster_to_id[i] = []
    
for i in range(len(prob_pair)):
    cluster_to_id[prob_pair[i][1]].append(prob_pair[i][0])

In [30]:
embedding = pd.read_csv("vectors_table")
embedding

,tactics_problem_id,1,2,3,4,5,6,7,8,9,...,256,257,258,259,260,261,262,263,264,265
0,731,0.019319,0.046485,0.001865,-0.035399,-0.072256,-0.056082,0.104733,0.135291,0.147258,...,0.031673,0.004133,0.026287,0.023505,0.108687,-0.075225,-0.027411,0.044459,0.087612,-0.028029
1,746,0.005904,0.025794,-0.030435,0.020898,0.079642,-0.094519,0.119855,-0.066765,-0.143177,...,0.075667,0.136880,0.049017,-0.015039,-0.003669,-0.026397,-0.056750,-0.049634,-0.026668,-0.018003
2,27811,-0.019099,0.026732,-0.045595,-0.026686,0.038413,-0.087863,0.093689,-0.060036,-0.127606,...,0.075912,0.167694,0.065821,-0.019764,-0.009593,0.031303,-0.060662,-0.017543,-0.024327,0.012986
3,905,-0.028095,0.068124,-0.049304,0.055087,0.062225,-0.052657,0.094387,-0.066688,-0.089172,...,0.141770,0.145417,0.052388,-0.015017,0.048582,0.030481,-0.083261,-0.049528,-0.026750,0.005764
4,48779,0.011405,0.033440,-0.058634,0.023071,0.077501,-0.060284,0.092497,-0.044495,-0.129376,...,0.066072,0.164938,0.041145,-0.019142,0.020974,0.013666,-0.056753,-0.040853,0.022477,-0.004774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42273,386259,-0.041488,-0.022765,-0.003912,0.029945,0.012708,-0.020448,0.137851,0.119394,0.066143,...,-0.008039,0.113656,-0.012311,-0.083416,0.142315,-0.024091,-0.012527,0.057587,-0.040969,-0.080014
42274,391319,-0.046158,-0.022334,-0.006139,0.033304,0.014259,-0.012193,0.130235,0.123793,0.060962,...,-0.017339,0.118149,-0.015816,-0.080210,0.146078,-0.019161,-0.021223,0.059119,-0.047276,-0.078951
42275,397091,-0.057911,-0.018623,0.005493,0.039696,0.007420,-0.010553,0.131130,0.119384,0.081336,...,-0.000231,0.139670,-0.002551,-0.080094,0.165461,-0.039162,-0.032764,0.073903,-0.064477,-0.066663
42276,408141,-0.055674,-0.024218,0.006734,0.047251,-0.004391,-0.022334,0.120599,0.125423,0.069912,...,0.001740,0.116558,-0.047480,-0.076570,0.161134,-0.014022,-0.049154,0.048670,-0.057606,-0.069933


In [36]:
puzz_id = list(table.columns)

avg_dist = {}

for cluster in range(1,21):
    puzzle_list = cluster_to_id[cluster]
    summation = 0
    count = 0
    for puzzle1 in puzzle_list:
        for puzzle2 in puzzle_list:
            pos1 = puzz_id.index(puzzle1)
            pos2 = puzz_id.index(puzzle2)
            dist = table.iloc[pos1,pos2]
            summation += dist
            count += 1
    avg_dist[cluster] = summation/count

avg_dist

{1: 0.17897907048322628,
 2: 0.445625,
 3: 0.4575302479338846,
 4: 0.42933545918367216,
 5: 0.40273701523545796,
 6: 0.4921824729891952,
 7: 0.50044,
 8: 0.5035876718034152,
 9: 0.5245240054869688,
 10: 0.4786326530612245,
 11: 0.35911111111111116,
 12: 0.4146666666666665,
 13: 0.4124444444444444,
 14: 0.5372704081632653,
 15: 0.49485260770975076,
 16: 0.49776388888888906,
 17: 0.5513089846603356,
 18: 0.5306419753086419,
 19: 0.49707438016528954,
 20: 0.42622509885535925}

In [38]:
summation = 0
count = 0
for puzzle1 in sampled_problems:
    for puzzle2 in sampled_problems:
        pos1 = puzz_id.index(puzzle1)
        pos2 = puzz_id.index(puzzle2)
        dist = table.iloc[pos1,pos2]
        summation += dist
        count += 1
baseline = summation / count
baseline

0.6765446880000097